### Exercice 1 : Initialisation de l'environnement et des structures de données ###

In [ ]:
import gymnasium as gym
import numpy as np

env = gym.make("Taxi-v3")

# Recuperation du nombre d'etats et d'actions
state_size = env.observation_space.n
action_size = env.action_space.n

# Création de la table de politique initialisé de maniere aleatoire 
policy_table = np.random.rand(state_size, action_size)
# Normaliser pour que chaque ligne soit une distribution de probabilité
policy_table = policy_table / policy_table.sum(axis=1, keepdims=True)

# Création de la table de valeurs initialisee à zero
value_table = np.zeros(state_size)

# Affichage des premières lignes des tables
print("Policy Table Sample:")
print(policy_table[:5])
print("Value Table Sample:")
print(value_table[:5])

Policy Table Sample:
[[0.0122697  0.23903713 0.21852465 0.20340184 0.17430298 0.15246369]
 [0.15271075 0.16408551 0.12788523 0.11005614 0.19794458 0.24731779]
 [0.12975878 0.12882445 0.14553763 0.21764171 0.03416063 0.3440768 ]
 [0.1393792  0.26441888 0.02753723 0.14168384 0.12006957 0.30691127]
 [0.00641275 0.31174619 0.02774712 0.23511643 0.38486477 0.03411274]]
Value Table Sample:
[0. 0. 0. 0. 0.]


### Exercice 2 : Exploration et collecte d'épisodes ###

In [10]:
num_episodes = 20
for episode in range(num_episodes):
    state, _ = env.reset()
    done = False
    while not done:
        action = env.action_space.sample()  # Choisir une action aléatoire
        next_state, reward, done, _, _ = env.step(action)
        print(f"Épisode {episode}, État {state}, Action {action}, Récompense {reward}")
        state = next_state


Épisode 0, État 87, Action 1, Récompense -1
Épisode 0, État 87, Action 1, Récompense -1
Épisode 0, État 87, Action 3, Récompense -1
Épisode 0, État 67, Action 5, Récompense -10
Épisode 0, État 67, Action 4, Récompense -10
Épisode 0, État 67, Action 4, Récompense -10
Épisode 0, État 67, Action 0, Récompense -1
Épisode 0, État 167, Action 1, Récompense -1
Épisode 0, État 67, Action 3, Récompense -1
Épisode 0, État 47, Action 3, Récompense -1
Épisode 0, État 47, Action 0, Récompense -1
Épisode 0, État 147, Action 1, Récompense -1
Épisode 0, État 47, Action 4, Récompense -10
Épisode 0, État 47, Action 0, Récompense -1
Épisode 0, État 147, Action 4, Récompense -10
Épisode 0, État 147, Action 4, Récompense -10
Épisode 0, État 147, Action 1, Récompense -1
Épisode 0, État 47, Action 4, Récompense -10
Épisode 0, État 47, Action 3, Récompense -1
Épisode 0, État 47, Action 1, Récompense -1
Épisode 0, État 47, Action 3, Récompense -1
Épisode 0, État 47, Action 1, Récompense -1
Épisode 0, État 47, 

### Exercice 3 : Mise à jour de la politique avec PPO ###

In [ ]:
# Hyperparamètres PPO
gamma = 0.99  # Facteur d'actualisation
lr_policy = 0.01  # Taux d'apprentissage pour la politique
lr_value = 0.01   # Taux d'apprentissage pour la fonction de valeur
clip_epsilon = 0.2  # Seuil de clipping

for episode in range(num_episodes):
    state, _ = env.reset()
    done = False
    
    episode_states = []
    episode_actions = []
    episode_rewards = []
    episode_probs = []
    
    while not done:
        # Sélection d'action selon la politique actuelle (avec échantillonnage)
        action_probs = policy_table[state]
        action = np.random.choice(action_size, p=action_probs)
        next_state, reward, done, _, _ = env.step(action)
        
        episode_states.append(state)
        episode_actions.append(action)
        episode_rewards.append(reward)
        episode_probs.append(action_probs[action])
        state = next_state
    
    # Calcul des recompenses cumulees (discounted rewards)
    discounted_rewards = np.zeros(len(episode_rewards))
    cumulative = 0
    for t in reversed(range(len(episode_rewards))):
        cumulative = episode_rewards[t] + gamma * cumulative
        discounted_rewards[t] = cumulative
    
    # Normalisation des avantages (pour stabilité)
    advantages = discounted_rewards - np.array([value_table[s] for s in episode_states])
    advantages = (advantages - advantages.mean()) / (advantages.std() + 1e-8)
    
    # Mise à jour de la politique avec clipping
    for t in range(len(episode_states)):
        state = episode_states[t]
        action = episode_actions[t]
        old_prob = episode_probs[t]
        advantage = advantages[t]
        
        # Calcul du ratio de probabilité
        current_prob = policy_table[state, action]
        ratio = current_prob / (old_prob + 1e-8)
        
        # Calcul des deux termes de l'objectif PPO
        policy_loss1 = ratio * advantage
        policy_loss2 = np.clip(ratio, 1 - clip_epsilon, 1 + clip_epsilon) * advantage
        
        # Maj de la politique
        policy_table[state, action] += lr_policy * np.minimum(policy_loss1, policy_loss2)
        
        # normalisation de la politique pour rester une distribution de probabilite
        policy_table[state] = np.clip(policy_table[state], 1e-8, 1-1e-8)
        policy_table[state] = policy_table[state] / policy_table[state].sum()
    
    # Mise à jour de la fonction de valeur
    for i, state in enumerate(episode_states):
        value_table[state] += lr_value * (discounted_rewards[i] - value_table[state])
    
    if (episode + 1) % 5 == 0:
        print(f"Épisode {episode+1}/{num_episodes} terminé. Récompense moyenne: {np.mean(episode_rewards)}")

### Exercice 4 : Évaluation de l'agent après entraînement ###

In [ ]:

# Évaluation de la politique apprise
num_eval_episodes = 20
total_rewards = []
for ep in range(num_eval_episodes):
    state, _ = env.reset()
    total_reward = 0
    done = False
    while not done:
        action = np.argmax(policy_table[state])
        state, reward, done, _, _ = env.step(action)
        total_reward += reward
    total_rewards.append(total_reward)
    
print(f"Récompense moyenne après entraînement: {np.mean(total_rewards)}")